# autologによるログの自動取得

これまでは記録するパラメータやアーティファクト等はこちらで指定したが、学習器(scikit-learn, XGBoost, LightGBM, TensorFlow, Keras etc)によっては、記録する情報を自動的に設定するautologという機能がある。

ここでは、scikit-learnにあるdiabetesというデータセットを通じて、scikit-learnの学習に対して`autolog()`を用いたとき、どのように記録されるかを確認する。  
また、最新のautologの実行結果を取得する`mlflow.last_active_run()`についても確認する。

In [1]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

In [2]:
db = load_diabetes()

In [3]:
X = db.data
y = db.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

In [5]:
mlflow.set_tracking_uri("http://localhost:5000")

In [6]:
# エクスペリメントの定義
experiment = mlflow.set_experiment("Auto_Logging")
experiment

2024/04/24 15:26:54 INFO mlflow.tracking.fluent: Experiment with name 'Auto_Logging' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/509013123844066675', creation_time=1713940014747, experiment_id='509013123844066675', last_update_time=1713940014747, lifecycle_stage='active', name='Auto_Logging', tags={}>

ここで`mlflow.autolog()`を実行することにより、自動ログ取得が実行される。

In [7]:
mlflow.autolog()

2024/04/24 15:26:54 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/04/24 15:26:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


このコンテキストを実行することにより、実際の学習・予測および記録がなされる。

In [8]:
with mlflow.start_run(run_name="trial autologging") as run:
    # Create and train models and predictions.
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

2024/04/24 15:26:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/user01/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."


記録が完了すると、Web UIより実験の実行処理の内容を確認することができるようになる。

一方で、直前の実行処理の内容は`mlflow.last_active_run()`にて取得可能である。

In [9]:
# Use the model to make predictions on the test dataset.
autolog_run = mlflow.last_active_run()
autolog_run

<Run: data=<RunData: metrics={'training_mean_absolute_error': 30.6991274607621,
 'training_mean_squared_error': 1382.797049433485,
 'training_r2_score': 0.7689636221432221,
 'training_root_mean_squared_error': 37.18597920498377,
 'training_score': 0.7689636221432221}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '6',
 'max_features': '3',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': 'None',
 'verbose': '0',
 'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
 'estimator_name': 'RandomForestRegressor',
 'mlflow.log-model.history': '[{"run_id": "e0d2dd63dbe24bf79e6d2930e8ad90ac", '
                             '"artifact_path": "model", "utc_time_created": '
